In [1]:
%reset -f
# Import functions from the module
from netcdf_tools import get_points, get_values

In [2]:
import pandas as pd

df = pd.read_csv('../data/trajectories.csv')

trajectories_df = pd.DataFrame(df)

trajectories_date = trajectories_df['date'].unique()


In [3]:
import glob as glob
import netCDF4 as nc

files = glob.glob('../data/wind/2014/01/*.nc')
print(len(files))
wind_data = nc.Dataset(files[30])
time_value = wind_data.variables['time'].actual_range[0]
date_time = nc.num2date(time_value, wind_data.variables['time'].units).strftime('%Y-%m-%d')
date_time

31


'2014-01-22'

In [4]:
daily_trajectories = get_points(trajectories_df, date_time)
daily_trajectories

,id,geom,date,latitude,longitude,name,season,spheroid_dist_to_colony,sst
1251,281,POINT(-118.291402 28.883883),2014-01-22,28.883883,-118.291405,LAAL01,incubacion,0.0,17.746
1252,282,POINT(-118.291442 28.883833),2014-01-22,28.883833,-118.291440,LAAL01,incubacion,0.0,17.746
1253,283,POINT(-118.291393 28.883812),2014-01-22,28.883812,-118.291390,LAAL01,incubacion,0.0,17.746
1254,284,POINT(-118.291473 28.883857),2014-01-22,28.883858,-118.291470,LAAL01,incubacion,0.0,17.746
1255,285,POINT(-118.29144 28.88382),2014-01-22,28.883820,-118.291440,LAAL01,incubacion,0.0,17.746
...,...,...,...,...,...,...,...,...,...
2254,92468,POINT(-118.291027 28.88367),2014-01-22,28.883670,-118.291020,LAAL14,incubacion,0.0,17.746
2255,92469,POINT(-118.291063 28.883687),2014-01-22,28.883686,-118.291060,LAAL14,incubacion,0.0,17.746
2256,92470,POINT(-118.29102 28.883648),2014-01-22,28.883648,-118.291020,LAAL14,incubacion,0.0,17.746
2257,92388,POINT(-118.290878 28.883568),2014-01-22,28.883568,-118.290880,LAAL14,incubacion,0.0,17.746


In [5]:
wind_data.variables.keys()

dict_keys(['time', 'altitude', 'latitude', 'longitude', 'wnd_ucmp_height_above_ground', 'wnd_vcmp_height_above_ground'])

In [6]:
import xarray as xr
import numpy as np

data = xr.open_dataset(files[0])
data.keys()


KeysView(<xarray.Dataset>
Dimensions:                       (time: 1, altitude: 1, latitude: 65,
                                   longitude: 107)
Coordinates:
  * time                          (time) datetime64[ns] 2014-01-14
  * altitude                      (altitude) float32 10.0
  * latitude                      (latitude) float32 23.0 23.5 ... 54.5 55.0
  * longitude                     (longitude) float32 -163.0 -162.5 ... -110.0
Data variables:
    wnd_ucmp_height_above_ground  (time, altitude, latitude, longitude) float32 ...
    wnd_vcmp_height_above_ground  (time, altitude, latitude, longitude) float32 ...
Attributes: (12/42)
    cdm_data_type:                        Grid
    Conventions:                          CF-1.6, COARDS, ACDD-1.3
    creator_email:                        CSO.navo.fct@navy.mil
    creator_name:                         FNMOC
    creator_type:                         institution
    creator_url:                          https://www.usno.navy.mil/FNMOC/

In [7]:
wind_units = data['wnd_ucmp_height_above_ground'].units
wind_date = data['time'].values
wind_date = pd.to_datetime(wind_date.astype('datetime64[s]')).strftime('%Y-%m-%d').values[0]

print(wind_units)
print(wind_date)

m s-1
2014-01-14


In [8]:
lon = data['longitude'].values
lat = data['latitude'].values
u_wind = data['wnd_ucmp_height_above_ground'].values
v_wind = data['wnd_vcmp_height_above_ground'].values

# Drop the height dimension
u_wind= np.squeeze(u_wind,axis=1)
v_wind= np.squeeze(v_wind,axis=1)
print(lat.shape, lon.shape, u_wind.shape, v_wind.shape)
# Print variable names
var_name = data['wnd_ucmp_height_above_ground'].name
print(var_name)

(65,) (107,) (1, 65, 107) (1, 65, 107)
wnd_ucmp_height_above_ground


In [11]:
u_wind_on_site = get_values(lon, lat, u_wind[0], var_name, daily_trajectories)
#v_wind_on_site = get_values(lon, lat, v_wind[0], daily_trajectories)
u_wind_on_site

,id,geom,date,latitude,longitude,name,season,spheroid_dist_to_colony,sst,wnd_ucmp_height_above_ground
1251,281,POINT(-118.291402 28.883883),2014-01-22,28.883883,-118.291405,LAAL01,incubacion,0.0,17.746,-2.9
1252,282,POINT(-118.291442 28.883833),2014-01-22,28.883833,-118.291440,LAAL01,incubacion,0.0,17.746,-2.9
1253,283,POINT(-118.291393 28.883812),2014-01-22,28.883812,-118.291390,LAAL01,incubacion,0.0,17.746,-2.9
1254,284,POINT(-118.291473 28.883857),2014-01-22,28.883858,-118.291470,LAAL01,incubacion,0.0,17.746,-2.9
1255,285,POINT(-118.29144 28.88382),2014-01-22,28.883820,-118.291440,LAAL01,incubacion,0.0,17.746,-2.9
...,...,...,...,...,...,...,...,...,...,...
2254,92468,POINT(-118.291027 28.88367),2014-01-22,28.883670,-118.291020,LAAL14,incubacion,0.0,17.746,-2.9
2255,92469,POINT(-118.291063 28.883687),2014-01-22,28.883686,-118.291060,LAAL14,incubacion,0.0,17.746,-2.9
2256,92470,POINT(-118.29102 28.883648),2014-01-22,28.883648,-118.291020,LAAL14,incubacion,0.0,17.746,-2.9
2257,92388,POINT(-118.290878 28.883568),2014-01-22,28.883568,-118.290880,LAAL14,incubacion,0.0,17.746,-2.9


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

u = u_wind[0,:,:]
v = v_wind[0,:,:]
magnitude = np.sqrt(u**2+v**2)

fig, axes=plt.subplots(figsize=(10,10))
axes.set_title('Wind Speed and Direction in the Pacific Ocean on {}'.format(date_time))
axes.set_xlabel('Longitude')
axes.set_ylabel('Latitude')
axes.set_xlim([-78.5,-77.5])
axes.set_ylim([-12.5,-11.5])


m=Basemap(projection='mill',lat_ts=15,llcrnrlon=lon.min(), \
          urcrnrlon=lon.max(),llcrnrlat=lat.min(),urcrnrlat=lat.max(), \
          resolution='i')

m.fillcontinents(color='0.6')
m.drawcoastlines()
x,y=m(*np.meshgrid(lon,lat))
#===============================
uproj,vproj,xx,yy=m.transform_vector(u,v,lon,lat,20,30,returnxy=True,masked=True)
#===============================

m.pcolormesh(x,y,magnitude,cmap=plt.cm.Reds)
m.quiver(xx,yy,uproj,vproj)
m.drawparallels(np.arange(-18,-2,1),labels=[1,0,0,0],linewidth=0.7,fontsize=9)
m.drawmeridians(np.arange(-86,-70,1),labels=[0,0,0,1],linewidth=0.7,fontsize=9)
m.colorbar()

# PLot the trajectories
x_1,y_1=m(daily_trajectories['longitude'].values,daily_trajectories['latitude'].values)
m.scatter(x_1, y_1, marker='o',color='b',s=10)

plt.show()